In [3]:
import numpy as np
import random
import operator 
import pandas as pd 
import matplotlib.pyplot as plt
import math

In [4]:
def routeFitness(route, dm):
    dist = 0
    for i in range(len(route)-1):
        dist += dm[route[i], route[i+1]]
    dist += dm[route[-1], route[0]]
    fitness = 1/dist
    
    return fitness

In [5]:
def dist_matrix(df):
    list_of_distances = [] # just to make it simple, probably there are 1k better ways to do this
    for point in df.itertuples():
        a = point[1]
        b = point[2]
        for point_s in df.itertuples():
            c = point_s[1]
            d = point_s[2]
            dist = math.sqrt(((c-a)**2) + ((d-b)**2))
            list_of_distances.append(dist)
    dist_matrix = np.array(list_of_distances) # convertion to array
    dist_matrix = np.reshape(dist_matrix, (len(df), len(df))) # reshaping
    # dist_matrix.shape # check
    return dist_matrix

In [6]:
def initPopulation(popSize, cities):
    population = []
    [population.append(random.sample(cities, len(cities))) for i in range(0, popSize)]
#     for i in range(0, popSize):
#         population.append(createRoute(cityList))
    return population

In [7]:
# bays29 opti 1610
# berlin52 opti 7542

df = pd.read_csv("berlin52.tsp", header=None, sep = " ", names = ['x', 'y'], index_col = 0)
df.head()
cities = []
cities = list(range(len(df)))
d_matrix = dist_matrix(df)

In [45]:
def GenA(permutation, dm, popSize, elite_n, mutation_r, n_of_gen, opti):
    
    pop = initPopulation(popSize, permutation)
    fitness = {}
    [fitness.update({i:routeFitness(pop[i], dm)}) for i in range(0,len(pop))]
    rankedRoutes = sorted(fitness.items(), key = operator.itemgetter(1), reverse = True)
    print("-------------------------------------")
    print("Initial distance: " + str(1 / rankedRoutes[0][1]))
    
    for k in range(0, n_of_gen):
        
        ##################
        # Ranking routes #
        ##################
        
        fitness = {}
        [fitness.update({i:routeFitness(pop[i], dm)}) for i in range(0,len(pop))]
        rankedRoutes = sorted(fitness.items(), key = lambda kv: kv[1], reverse = True)

        #############
        # selection #
        #############
        
        selectionR = []
        df = pd.DataFrame(np.array(rankedRoutes), columns=["Index","Fitness"])
        df['cum_sum'] = df.Fitness.cumsum()
        df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()
        
        [selectionR.append(rankedRoutes[i][0]) for i in range(0, elite_n)]
        
        for i in range(0, len(rankedRoutes) - elite_n):
            pick = 100*random.random()
            for i in range(0, len(rankedRoutes)):
                if pick <= df.iat[i,3]:
                    selectionR.append(rankedRoutes[i][0])
                    break  
        
        ###############
        # mating pool #
        ###############
        
        mp = []
        [mp.append(pop[selectionR[i]]) for i in range(0, len(selectionR))]

        #############
        # Crossover #
        #############
        
        children = []
        length = (len(mp) - elite_n)
        pool = random.sample(mp, len(mp))
        [children.append(mp[i]) for i in range(0, elite_n)]
        
        for i in range(0, length):
            parent1 = pool[i]
            parent2 = pool[len(mp)-i-1]
            child = []
            childP1 = []
            childP2 = []
    
            geneA = int(random.random() * len(parent1))
            geneB = int(random.random() * len(parent1))
            
            startGene = min(geneA, geneB)
            endGene = max(geneA, geneB)
            [childP1.append(parent1[i]) for i in range(startGene, endGene)]
            childP2 = [item for item in parent2 if item not in childP1]
            
            child = childP1 + childP2
            children.append(child)
        
        ############
        # mutation #
        ############
        
        mutatedPop = []
    
        for ind in range(0, len(children)):
            individual = children[ind]
            for swapped in range(len(individual)):
                if(random.random() < mutation_r):
                    swapWith = int(random.random() * len(individual))
            
                    city1 = individual[swapped]
                    city2 = individual[swapWith]
            
                    individual[swapped] = city2
                    individual[swapWith] = city1
            mutatedPop.append(individual)
        
        ##############
        # update pop #
        ##############
        
        pop = mutatedPop
        
    
    
    ###########
    # results #
    ###########
    
    fitness = {}
    [fitness.update({i:routeFitness(pop[i], dm)}) for i in range(0,len(pop))]
    rankedRoutes = sorted(fitness.items(), key = lambda kv: kv[1], reverse = True)
    print("#####################################")
    print("Final distance: " + str(1 / rankedRoutes[0][1]))
    print("#####################################")
    result = str(((1/rankedRoutes[0][1] - opti)/opti)*100)
    print("Solution is {} of optimum".format(str(((1/rankedRoutes[0][1] - opti)/opti)*100)))
    print("-------------------------------------")

In [46]:
GenA(permutation = cities, popSize = 200, dm = d_matrix, elite_n = 30, mutation_r = 0.01, n_of_gen = 100, opti = 7542)

-------------------------------------
Initial distance: 25540.787412884998
#####################################
Final distance: 15814.569493562243
#####################################
Solution is 109.68668116629863 of optimum
-------------------------------------
